In [ ]:
#standard
from dotenv import load_dotenv
load_dotenv()
import os
import sys
sys.path.append(os.getenv('PYTHONPATH')) 
import warnings
warnings.filterwarnings('ignore')
import pickle

#third party
import numpy as np
import hcp_utils as hcp
import matplotlib.pyplot as plt
from nilearn import plotting

#local
from src.utils.helpers import ComputeNoiseceiling

In [ ]:
#setup paths
datasets_root = os.path.join(os.getenv("DATASETS_ROOT", "/default/path/to/datasets")) #use default if DATASETS_ROOT env variable is not set.
dataset_root = os.path.join(datasets_root,"deeprecon")
meta_dataset_root = os.path.join(datasets_root, "MOSAIC")

project_root = os.getenv("PROJECT_ROOT", "/default/path/to/project")
print(f"dataset_root: {dataset_root}")
print(f"project_root: {project_root}")
fmri_path = os.path.join(dataset_root,"derivatives", "GLM") #change folder to "GLM_individualsession" to compare the glmsingle typeb version

In [ ]:
ext_list = ['png'] #only matters if save_flag is True 
n_task = {'train': [1,'avg'], 'test': [1, 'avg'], 'artificial': [1,'avg']} #repeats are 24, 20 and 12 for naturalTest, shapes, and letters, respectively
dataset = 'deeprecon'
save_flag= True #set to True to save plots or False to not save plots
for subject in range(1,4):
    save_root = os.path.join(project_root, "src", "fmriDatasetPreparation", "datasets", "deeprecon", "validation", "output", "noiseceiling", f"sub-{subject:02}")
    if not os.path.exists(save_root):
        os.makedirs(save_root)
    for task in n_task.keys():
        with open(os.path.join(fmri_path, f"sub-{subject:02}", "prepared_betas", f"sub-{subject:02}_organized_betas_task-{task}_normalized.pkl"), 'rb') as f: 
            betas, stimorder = pickle.load(f) #betas is shape numstim, numreps, numvertices
        betas = betas.T
        print(betas.shape)
        for n in n_task[task]:
            print("calculating noiseceiling...")
            ncsnr, noiseceiling = ComputeNoiseceiling(betas, n=n).compute_ncsnr_noiseceiling()
            print(f"sub-{subject:02} {task} n={n} max noiseceiling: {np.nanmax(noiseceiling)}")
            
            noiseceiling[noiseceiling < 0] = 0
            if save_flag:
                np.save(os.path.join(save_root, f"sub-{subject:02}_{dataset}_phase-{task}_n-{n}_noiseceiling.npy"), noiseceiling)
                np.save(os.path.join(meta_dataset_root, "noiseceilings", f"sub-{subject:02}_{dataset}_phase-{task}_n-{n}_noiseceiling.npy"), noiseceiling)
            stat = noiseceiling.copy()

            #save inflated surfaces
            views = [] #['lateral', 'medial'] #['lateral', 'medial', 'dorsal', 'ventral', 'anterior', 'posterior']
            for hemi in ['left','right']:
                mesh = hcp.mesh.inflated
                cortex_data = hcp.cortex_data(stat)
                bg = hcp.mesh.sulc
                for view in views:
                    display = plotting.plot_surf_stat_map(mesh, cortex_data, hemi=hemi,
                    threshold=1, bg_map=bg, view=view, cmap='hot')
                    if save_flag:
                        for ext in ext_list:
                            if ext == 'png':
                                plt.savefig(os.path.join(save_root, f"sub-{subject:02}_{dataset}_task-{task}_mesh-inflated_view-{view}_hemi-{hemi}_n-{n}_noiseceiling.{ext}"),dpi=300)
                            else:
                                plt.savefig(os.path.join(save_root, f"sub-{subject:02}_{dataset}_task-{task}_mesh-inflated_view-{view}_hemi-{hemi}_n-{n}_noiseceiling.{ext}"))

            #Save flat maps. hemispheres are combined in one plot
            #get the data for both hemispheres
            cortex_data_left = hcp.left_cortex_data(stat)
            cortex_data_right = hcp.right_cortex_data(stat)

            #determine global min/max for consistent color scaling
            datamin = min(np.nanmin(cortex_data_left), np.nanmin(cortex_data_right))
            datamax = max(np.nanmax(cortex_data_left), np.nanmax(cortex_data_right))
            vmin=0
            vmax=100

            #create a figure with multiple axes to plot each anatomical image
            fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 4), subplot_kw={'projection': '3d'})
            plt.subplots_adjust(wspace=0)
            im = plotting.plot_surf(hcp.mesh.flat_left, cortex_data_left,
                    threshold=1, bg_map=hcp.mesh.sulc_left, 
                    colorbar=False, cmap='hot', 
                    vmin=vmin, vmax=vmax,
                    axes = axes[0])
            im = plotting.plot_surf(hcp.mesh.flat_right, cortex_data_right,
                    threshold=1, bg_map=hcp.mesh.sulc_right, 
                    colorbar=False, cmap='hot', 
                    vmin=vmin, vmax=vmax,
                    axes = axes[1])
            
            #flip along the horizontal
            axes[0].invert_yaxis()
            axes[1].invert_yaxis()

            #create colorbar
            norm = plt.Normalize(vmin=vmin, vmax=vmax)
            sm = plt.cm.ScalarMappable(cmap='hot', norm=norm)
            sm.set_array([])
            cbar = fig.colorbar(sm, ax=axes.ravel().tolist(), shrink=0.6)

            cbar.set_ticks([0, round(datamax), 100])
            cbar.set_ticklabels([0, round(datamax), 100])
            if save_flag:
                for ext in ext_list:
                    if ext == 'png':
                        plt.savefig(os.path.join(save_root, f"sub-{subject:02}_{dataset}_task-{task}_mesh-flat_n-{n}_noiseceiling.{ext}"),dpi=300)
                    else:
                        plt.savefig(os.path.join(save_root, f"sub-{subject:02}_{dataset}_task-{task}_mesh-flat_n-{n}_noiseceiling.{ext}"))
            plt.show()
